In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [25]:
training = pd.read_csv("criminal_train.csv", index_col='PERID')

In [46]:
# Parameters
training_epochs = 20
batch_size = 9
start_rate = 0.0002

# Network Parameters
n_hidden_1 = 35 # 1st layer number of neurons
n_hidden_2 = 50 # 2nd layer number of neurons
n_features = 70 # 
n_classes = 2 #

########### Defining tensorflow computational graph ###########

# tf Graph input
# Features
X = tf.placeholder(tf.float32, [None, n_features])
# Labels
Y = tf.placeholder(tf.float32, [None,1])
# decay step for learning rate decay
decay_step = tf.placeholder(tf.int32)


# Create model
def deep_neural_network(x):

    # Hidden fully connected layer with 100 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1, activation=tf.nn.relu)
    # Hidden fully connected layer with 50 neurons
    #layer_2 = tf.layers.dense(layer_1, n_hidden_2, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_1, 1)
    return tf.sigmoid(out_layer)

# Construct model
logits = deep_neural_network(X)

# Define loss and optimizer
# Converting categories into one hot labels
labels = Y
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    					logits=logits, labels=labels))
global_step = tf.Variable(0, trainable=False)

# Using a learning rate which has polynomial decay
starter_learning_rate = start_rate
end_learning_rate = 0.00005 # we will use a polynomial decay to reach learning this learning rate.29
decay_steps = decay_step
learning_rate = tf.train.polynomial_decay(starter_learning_rate, global_step,
                                          decay_steps, end_learning_rate,
                                          power=0.5)
# Using adam optimizer to reduce loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing the variables
init = tf.global_variables_initializer()

# Model for testing
prediction = logits # 



In [37]:
cols_to_normalise = list(training.columns.values)[:70]
training[cols_to_normalise] = training[cols_to_normalise].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [47]:
def get_accuracy(df):

	# Calculate accuracy for label classification
	preds = prediction.eval(feed_dict={X: df.iloc[:, 0:-1]}) # Getting back the predictions


	# Preparing Labels
	labels = df.iloc[:, -1].values
	
	# Compating predictions and labels to calculate accuracy
	correct_labels = tf.equal(preds, labels)
	label_accuracy = tf.reduce_mean(tf.cast(correct_labels, tf.float32))
	result = label_accuracy.eval()
	print("Label accuracy: {:.2f}%".format(result*100))


In [ ]:
def train_and_test_model(training, testing):
    with tf.Session() as sess:
        sess.run(init)

        # Training cycle
        for epoch in range(training_epochs):
            # Shuffling dataset before training
            df = training.sample(frac=1)
            avg_cost = 0.
            total_data = df.index.shape[0] 
            num_batches = total_data // batch_size + 1
            i = 0
            # Loop over all batches
            while i < total_data:
                batch_x = df.iloc[i:i+batch_size, 0:-1].values
                batch_y = df.iloc[i:i+batch_size, -1].values # Last column is labels
                batch_y = batch_y.reshape(batch_y.shape[0], 1)
                i += batch_size
                # Run optimization op and cost op (to get loss value)
                _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                                Y: batch_y,
                                                                decay_step: num_batches * training_epochs})
                # Compute average loss
                avg_cost += c / num_batches
            # Display logs per epoch step
            print("Epoch: {:04} | Cost={:.9f}".format(epoch+1, avg_cost))
            get_accuracy(testing)
            print()
        print("Training complete")

        print("Training results: ")
        get_accuracy(training)
        print("Testing results: ")
        get_accuracy(testing)


# Training the model after shuffling the data.
train_and_test_model(training, training)
